In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 4 – Hypothesis and Permutation Testing

## DSC 80, Fall 2024

### Due Date: Monday, October 25th at 11:59PM

## Instructions

Welcome to the fourth DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2024-fa).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:
from lab import *

In [5]:
import pandas as pd
import numpy as np
import io
from pathlib import Path
import os

<div class="alert alert-block alert-danger" markdown="1">

**You cannot use `for`-loops in Part 1 (Time Series Data) or Part 2 (Hypothesis Testing), but you can use them in Part 3 (Permutation Testing).**

</div>

## Part 1: Time Series Data

Imagine that you own an online store and you'd like to monitor the visits to your site. You've collected information about different login dates and times for different users and stored it in `data/login_table.csv`. Some users are unique, while some visited your store multiple times.

Answer the questions below to better understand the login patterns of your users.

### Question 1 – Prime Time ⏰

Complete the implementation of the function `prime_time_logins`, which takes in a DataFrame like `login` and outputs a DataFrame indexed by `'Login Id'`, counting the number of prime-time logins for each user – that is, the number of logins that were between 4PM (inclusive) and 8PM (exclusive) for each user. The DataFrame should have just one column, named `'Time'`.

For example, if a user logs in at 5PM on Day 1, at 1PM on Day 2, at 6PM on Day 2, and at 7PM on Day 2, then their total number of prime-time logins is 3. Note that the values in your returned DataFrame should only include counts, not timestamp objects.

***Note***: You do not need to use Python's `datetime` module – instead, use the built-in `pandas` methods for working with times that we introduced in [Lecture 5](https://dsc80.com/resources/lectures/lec05/lec05.html#Creating-timestamps) (though you may need to do a bit more research to fully answer the question).

In [6]:
# def prime_time_logins(login_df):
#     '''
    
#     after set index to login id?
#     do a broadcast comparsion of times in a range
#     4 up 8pm

#     date_format = '%Y-%m-%d'
#     pd.to_datetime(insp['date'], format=date_format)
    
#     '''
#     # Set 'Login Id' as the index
#     login_df = login_df.set_index('Login Id')


#     login_df['Time'] = pd.to_datetime(login_df['Time'], format='%Y-%m-%d %H:%M:%S')
#     # Extract the hour component from the datetime values
#         # NOTICE : this won't work without the explicit date_format made above
#     hours = login_df['Time'].dt.hour
    
#     # Create a mask for prime-time logins (4PM to 8PM)
#     prime_time_mask = (hours >= 16) & (hours < 20)
    
#     # Count prime-time logins for each user
#     prime_time_count = login_df[prime_time_mask].groupby(level=0).size().rename('Time')
    
#     return prime_time_count
    
    

In [7]:

def prime_time_logins(login_df):
    '''
    Takes in a DataFrame with 'Login Id' and 'Time' columns,
    and returns a DataFrame with the count of prime-time logins for each user.
    Prime-time is defined as between 4PM (inclusive) and 8PM (exclusive).
    '''
    # Make a copy of the DataFrame to avoid modifying the original
    login_df = login_df.copy()

    # Convert 'Time' column to datetime
    login_df['Time'] = pd.to_datetime(login_df['Time'], format='%Y-%m-%d %H:%M:%S')
    
    # Extract the hour component from the datetime values
    hours = login_df['Time'].dt.hour
    
    # Create a mask for prime-time logins (4PM to 8PM)
    prime_time_mask = (hours >= 16) & (hours < 20)
    
    # Count prime-time logins for each user
    prime_time_count = login_df[prime_time_mask].groupby('Login Id').size().to_frame('Time')
    
    # Ensure all users are included, even those with zero prime-time logins
    all_users = login_df['Login Id'].unique()
    prime_time_count = prime_time_count.reindex(all_users, fill_value=0)
    
    return prime_time_count

In [8]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'login_table.csv'
login = pd.read_csv(fp)
q1_result = prime_time_logins(login)

In [9]:
print(q1_result.loc[457])

Time    4
Name: 457, dtype: int64


In [10]:
prime_time_logins(login)

,Time
Login Id,
466,38
458,52
592,38
393,12
636,0
...,...
1302,2
1304,0
1305,1


In [11]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – Return Users 🔁

As a site owner, you would like to find your most enthusiastic users – the ones who return to your site most frequently. You've noticed that there are users who have several logins and users who logged in only once. You are interested in finding the number of logins *per day* for each user – that is, **their total number of logins divided by the total number of days they've been on your site**.

Complete the implementation of the function `count_frequency`, which takes in a DataFrame like `login` and outputs a Series containing the number of logins per day for each user. Your Series should have `'Login Id'`s in its index, and the frequencies as its values. The order of users in the index is arbitrary.

To do this, you should assume today is January 31st, 2024 and use 11:59:00PM as the current time. The first login date of a user is the first day of their membership on the site, and you may assume they are still a member today. For simplicity, you only need to count full days that a user has been a member till the end of today. For example, if a user's first login was 12 days and 5 hours ago, you can say that they have been a user for 12 days. 

***Hint***: Can you write a custom aggregator that allows you to do this with just one call of the `groupby` method?

In [12]:
# def count_frequency(login_df):
#     '''
#     use date time like early same format
#     make a conversion of the date time to quantity of days
    
    
#     '''
#     # Make a copy of the DataFrame to avoid modifying the original
#     login_df = login_df.copy()
    
#     # Convert 'Time' column to datetime if it's not already
#     login_df['Time'] = pd.to_datetime(login_df['Time'])
    
#     # Set the end date (January 31st, 2024 at 11:59:00 PM)
#     end_date = pd.Timestamp('2024-01-31 23:59:00')
    
#     # Define a custom aggregation function
#     def login_frequency(group):
#         first_login = group.min()
#         total_logins = group.count()
#         days_as_member = (end_date - first_login).days + 1  # +1 to include the last day
#         return total_logins / days_as_member
    
#     # Group by 'Login Id' and apply the custom aggregation
#     result = login_df.groupby('Login Id')['Time'].agg(login_frequency)
#     return result
    

In [13]:

# def count_frequency(login_df):
#     """
#     Calculate the number of logins per day for each user.
    
#     Parameters:
#     -----------
#     login_df : pandas.DataFrame
#         DataFrame containing 'Login Id' and 'Time' columns
        
#     Returns:
#     --------
#     pandas.Series
#         Series with Login Ids as index and login frequency (logins per day) as values
#     """
#     # Make a copy of the DataFrame to avoid modifying the original
#     df = login_df.copy()
    
#     # Convert 'Time' column to datetime if it's not already
#     df['Time'] = pd.to_datetime(df['Time'])
    
#     # Set reference end date (January 31st, 2024 at 11:59:00 PM)
#     end_date = pd.Timestamp('2024-01-31 23:59:00')
    
#     def calculate_frequency(group):
#         """
#         Custom aggregator function to calculate login frequency
#         """
#         # Get first login date and total number of logins
#         first_login = group.min()
#         total_logins = len(group)
        
#         # Calculate total days (including the first and last day)
#         days_as_member = (end_date - first_login).days + 1
        
#         # Calculate and return frequency
#         return total_logins / days_as_member
    
#     # Group by Login Id and apply the custom aggregator
#     result = df.groupby('Login Id')['Time'].agg(calculate_frequency)
    
#     return result





In [14]:
def count_frequency(login_df):
    # Make a copy of the DataFrame to avoid modifying the original
    login_df = login_df.copy()
    
    # Convert 'Time' column to datetime
    login_df['Time'] = pd.to_datetime(login_df['Time'])
    
    # Set the end date to January 31st, 2024, at 11:59 PM
    end_date = pd.Timestamp('2024-01-31 23:59:00')
    
    # Define a custom aggregation function to calculate login frequency
    def login_frequency(group):
        first_login = group.min()  # First login date per user
        total_logins = group.size  # Use 'size' as a property, not a method
        days_as_member = (end_date - first_login).days  # Days as member (inclusive)
        if days_as_member == 0:
            return total_logins  # Handle users with logins only on one day
        return total_logins / (days_as_member)  # +1 to include the last day
    
    # Group by 'Login Id' and apply the custom aggregation
    result = login_df.groupby('Login Id')['Time'].agg(login_frequency)
    return result

In [15]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'login_table.csv'
login = pd.read_csv(fp)
q2_result = count_frequency(login)

In [16]:
q2_result.loc[466]

np.float64(0.24250681198910082)

In [17]:
grader.check("q2")

q2 results: All test cases passed!

<div class="alert alert-block alert-danger" markdown="1">

**You cannot use `for`-loops in Part 1 (Time Series Data) or Part 2 (Hypothesis Testing), but you can use them in Part 3 (Permutation Testing).**

</div>

## Part 2: Hypothesis Testing

In this section, you'll recall the terms and structure of hypothesis testing from DSC 10.

The first step is always to define what you're looking at, create your hypotheses, and set a level of significance (i.e. a p-value cutoff). Once you've done that, you can find a p-value.

If all of these words are foreign, look at the [Lecture 6](https://dsc80.com/resources/lectures/lec06/lec06.html) notebook and the readings, and don't forget to think about the real-world meaning of these terms!  The following example describes a real-world scenario, which should help keep it easy to interpret.

### Question 3 – Baking Sale 🧁

At UC San Diego, students are looking to buy treats at a bake sale on Library Walk. There is a pop-up bake stand selling cookies and cupcakes to students. Last Saturday, this stand sold 250 cookies to UCSD students. After eating the cookies, 15 students complained that their cookies were burnt, leaving a bitter taste in their mouths. In response to the student dissatisfaction, the stand claims that 96% of their cookies are baked perfectly without any burning. You think this seems unlikely and decide to investigate.

First, select a significance level for your investigation. You don't need to turn this in anywhere. Then, complete the implementation of the following two functions.

#### `cookies_null_hypothesis`

Complete the implementation of the function `cookies_null_hypothesis`, takes in no arguments and returns your answer(s) to the following question **as a list**.

What are reasonable choices for the **null hypothesis** for your investigation? Select all that apply.
1. The stand sells cookies that are approximately 4% burnt. 
2. The stand sells cookies that are 96% perfectly baked.
3. The stand sells cookies that are less than 96% perfectly baked. 
4. The stand sells cookies that are at least 4% burnt.

<br>

#### `cookies_p_value`

Complete the implementation of the function `cookies_p_value`, which takes in an integer `N` and returns the estimated p-value of your investigation upon simulating the null hypothesis `N` times. (The p-value is an estimate of the true theoretical p-value of your test since it relies on simulation.)

***Note***: Plot the null distribution and your observed statistic to check your work. (If you decide to plot, you may have to run `import matplotlib.pyplot as plt` or `import plotly.express as px`.)

<h1>why was this code not working and expecting a specific set?

In [18]:
# def cookies_null_hypothesis():
#     """
#     Returns a list of reasonable choices for the null hypothesis.
#     """
#     return [
#         "The stand sells cookies that are approximately 4% burnt.",
#         "The stand sells cookies that are 96% perfectly baked.",
#         "The stand sells cookies that are at least 96% perfectly baked."
#     ]




# def cookies_p_value(N_simulations):
#     """
#         GOAL get the tests onesided and two sided correctl with 96% of cookies are NOT burnt
#         claim
    
#     """
#     # Define the parameters of the null hypothesis
#     n_cookies = 250
#     p_burnt = 0.04
    
#     # Simulate the null hypothesis N times
#     simulations = np.random.binomial(n_cookies, p_burnt, N_simulations)
    
#     # Calculate the observed statistic (number of burnt cookies)
#     observed_statistic = 15
    
#     # Calculate the p-value (one-tailed test)
#     p_value = np.mean(simulations >= observed_statistic)
    
#     return p_value

In [19]:

def cookies_null_hypothesis():
    """
    Returns a list of reasonable choices for the null hypothesis.
    """
    return [1, 2]

def cookies_p_value(N_simulations):
    """
    Estimates the p-value of the investigation by simulating the null hypothesis N times.
    
    Parameters:
    N_simulations (int): The number of simulations to run.
    
    Returns:
    float: The estimated p-value.
    """
    # Define the parameters of the null hypothesis
    n_cookies = 250
    p_burnt = 0.04
    
    # Simulate the null hypothesis N times
    simulations = np.random.binomial(n_cookies, p_burnt, N_simulations)
    
    # Calculate the observed statistic (number of burnt cookies)
    observed_statistic = 15
    
    # Calculate the p-value (one-tailed test)
    p_value = np.mean(simulations >= observed_statistic)
    
    return float(p_value)


In [20]:
grader.check("q3")

q3 results: All test cases passed!

Now that we've gotten our feet wet with hypothesis testing, let's take a closer look at how to choose null and alternative hypotheses and test statistics.

### Question 4 – Tires 🚗

A tire manufacturer, TritonTire, claims that their tires are so good, they will bring a Toyota Highlander from 60 MPH to a complete stop in under 106 feet, 95% percent of the time.

Now, you own a Toyota Highlander equipped with TritonTire tires, and you decide to test this claim. You take your car to an empty Vons parking lot, speed up to exactly 60 MPH, hit the brakes, and measure the stopping distance. As illegal as it is, you repeat this process 50 times and find that **you stopped in under 106 feet only 47 of the 50 times**.

Livid, you call TritonTire and say that their claim is false. They say, no, that you were just unlucky: your experiment is consistent with their claim. But they didn't realize that they are dealing with a *data scientist* 🧑‍🔬.

To settle the matter, you decide to unleash the power of the hypothesis test. The following three subparts ask you to answer a total of four select-all multiple choice questions.

#### Question 4.1: `car_null_hypothesis` and `car_alt_hypothesis`

You will set up a hypothesis test in order to test your suspicion that the tires are are actually worse than claimed. Which of the following are valid null and alternative hypotheses for this hypothesis test?

1. The tires will stop your car in under 106 feet exactly 95% of the time.
0. The tires will stop your car in under 106 feet less than 95% of the time.
0. The tires will stop your car in under 106 feet greater than 95% of the time.
0. The tires will stop your car in more than 106 feet exactly 5% of the time.
0. The tires will stop your car in more than 106 feet less than 5% of the time.
0. The tires will stop your car in more than 106 feet greater than 5% of the time.

Complete the implementation of the function `car_null_hypothesis`, which takes zero arguments and returns a list of integers, corresponding to the the valid null hypotheses above.
Also complete the implementation of the function called `car_alt_hypothesis`, which takes zero arguments and returns a list of integers, corresponding to the valid alternative hypotheses above given your observation.

<br>

#### Question 4.2: `car_test_statistic`

Which of the following are valid test statistics for our question?

1. The number of times the car stopped in under 106 feet in 50 attempts.
1. The average number of feet the car took to come to a complete stop in 50 attempts.
1. The number of attempts it took before the car stopped in under 95 feet.
1. The proportion of attempts in which the car stopped in under 106 feet in 50 attempts.

Complete the implementation of the function `car_test_stat`, which takes zero arguments and returns a list of integers, corresponding to the valid test statistics above.

<br>

#### Question 4.3: `car_p_value`

The p-value is the probability, under the assumption the null hypothesis is true, of observing a test statistic **equal to our observed statistic, or more extreme in the direction of the alternative hypothesis**.

Why don't we just look at the probability of observing a test statistic equal to our observed statistic? That is, why is the "more extreme in the direction of the alternative hypothesis" part necessary?

1. Because our observed test statistic isn't extreme.
2. Because our null hypothesis isn't suggesting equality.
3. Because the probability of finding our observed test statistic equals the probability of finding something more extreme.
4. Because if we run more and more trials, the probability of observing any particular test statistic gets closer and closer to zero.

Complete the implementation of the function `car_p_value`, which takes zero arguments and returns the correct reason as an **integer** (not a list).

In [21]:
# def car_null_hypothesis():
#     # The null hypothesis is that the tires stop the car in under 106 feet exactly 95% of the time.
#     return [0]

# def car_alt_hypothesis():
#     # The alternative hypothesis is that the tires stop the car in under 106 feet less than 95% of the time.
#     return [1]

# def car_test_stat():
#     # The valid test statistics are the number of times and the proportion of attempts.
#     return [0, 3]

# def car_p_value():
#     # The reason is that the probability of observing a specific test statistic decreases with more trials.
#     return 4

In [22]:
# def car_null_hypothesis():
#     # The claim is that tires stop under 106 feet exactly 95% of the time
#     return [1]  # Option 1

# def car_alt_hypothesis():
#     # We suspect the success rate is less than 95%
#     return [2]  # Option 2

# def car_test_stat():
#     # Both count and proportion are valid test statistics
#     return [1, 4]  # Options 1 and 4

# def car_p_value():
#     # The probability of any exact value becomes tiny with more trials
#     return 4  # Option 4

In [23]:
def car_null_hypothesis():
    # The claim is that tires stop under 106 feet exactly 95% of the time
    return [1]  # Option 1

def car_alt_hypothesis():
    # We suspect the success rate is less than 95%
    return [2]  # Option 2

def car_test_statistic():  # Fixed function name to match test case
    # Both count and proportion are valid test statistics
    return [1, 4]  # Options 1 and 4

def car_p_value():
    # The probability of any exact value becomes tiny with more trials
    return 4  # Option 4

In [24]:
grader.check("q4")

q4 results: All test cases passed!

### Question 5 – Superheroes 🦸

In the previous two questions, we ran hypothesis tests that didn't require us to look at stored data. In this next question, we'll return to the `heroes` DataFrame from Lab 2, which is read in from the file `data/superheroes.csv`.

Our goal in this section will be to answer the question:

> Are there significantly **more** "good" blond-haired, blue-eyed characters than the general pool of characters?

To answer this question, we will conduct a hypothesis test. You choose the following null hypothesis:

> The proportion of "good" characters among blond-haired, blue-eyed characters is equal to the proportion of "good" characters in the sample population from the current `heroes` DataFrame.

and alternative hypothesis: 

> the distribution of "good" characters among blond-haired, blue-eyed characters is greater than the proportion of "good" characters in the overall population.

To proceed with the hypothesis test, we will need to determine the test statistics for our test.

#### `superheroes_test_statistic`

Which of the following are valid test statistics for our question?

1. The difference in proportions for "good" characters among blond-haired, blue-eyed characters and "good" characters in the overall population. 
1. The number of "good" characters that are blond-haired, blue-eyed.
1. The proportion of blond-haired, blue-eyed characters among all "good" characters.
1. The absolute difference in proportions for "good" characters among blond-haired, blue-eyed characters and "good" characters in the overall population.

Complete the implementation of the function `superheros_test_stat`, which takes zero arguments and returns a list of integers, corresponding to all the valid test statistics above.

---

Regardless of your choice for the above question, we will use the test statistic stated below to complete the implementations of the following functions:

> The proportion of "good" characters among blond-haired, blue-eyed characters.

#### `bhbe_col`

To start, complete the implementation of the function `bhbe_col`, which takes in a DataFrame like `heroes` and returns a Boolean Series that contains `True` for characters that have **both** blond hair and blue eyes, and `False` for all other characters. 

***Note***: If a character's hair color contains the word `'blond'`, uppercase or lowercase, we consider their hair to be blond for the purposes of this question. Similarly, if a character's eye color contains the word `'blue'`, uppercase or lowercase, we consider their eye color to be blue for the purposes of this question.

Fix a significance level (i.e. p-value cutoff) of 1%.

<br>

#### `superheroes_observed_statistic`
Complete the implementation of the function `superheroes_observed_statistic`, which takes in a DataFrame like `heroes` and returns the observed test statistic.

<br>

#### `simulate_bhbe_null` 
Complete the implementation of the function `simulate_bhbe_null`, which takes in a DataFrame like `heroes` and a positive integer `N` and returns an array of length `N`, where each element is a simulated test statistic according to the null hypothesis.

***Hint***: Like in `superheroes_observed_statistic`, you'll need to use both `bhbe_col` and information in the `heroes` DataFrame to complete your simulation. Remember that you cannot use `for`-loops in this question.

<br>

#### `superheroes_p_value` 
Complete the implementation of the function `superheroes_p_value`, which takes in DataFrame like `heroes` and returns a list where:
* The first element is the p-value for the hypothesis test, using 100,000 simulations.
* The second element is `'Reject'` if you reject the null hypothesis and `'Fail to reject'` if you fail to reject the null hypothesis, at the 1% significance level.

In [25]:
# def superheroes_p_value:
#     return
# def simulate_bhbe_null:
#     return
# def superheroes_observed_statistic:
#     return
# def bhbe_col:
#     return
# def superheroes_test:
#     return

In [26]:
def superheroes_test_statistic():
    return [1, 2, 4]
def bhbe_col(df):
    return (df['Hair color'].str.contains('blond', case=False)) & (df['Eye color'].str.contains('blue', case=False))
def superheroes_observed_statistic(df):
    bhbe = bhbe_col(df)
    return (df['Alignment'] == 'good')[bhbe].mean()

def simulate_bhbe_null(df, N):
    bhbe = bhbe_col(df)
    good_proportion = (df['Alignment'] == 'good').mean()
    simulated_stats = np.random.binomial(n=bhbe.sum(), p=good_proportion, size=N) / bhbe.sum()
    return simulated_stats

def superheroes_p_value(df):
    observed_stat = superheroes_observed_statistic(df)
    simulated_stats = simulate_bhbe_null(df, 100000)
    p_value = (simulated_stats >= observed_stat).mean()
    if p_value < 0.01:
        return [p_value, 'Reject']
    else:
        return [p_value, 'Fail to reject']




In [27]:
def superheroes_test_statistic():
    # The valid test statistics are:
    # 1. Difference in proportions (Option 1)
    # 4. Absolute difference in proportions (Option 4)
    return [1, 4]

def bhbe_col(df):
    """Return a Boolean Series for blond-haired, blue-eyed characters."""
    return (df['Hair color'].str.contains('blond', case=False, na=False)) & \
           (df['Eye color'].str.contains('blue', case=False, na=False))

def superheroes_observed_statistic(df):
    """Calculate the observed proportion of 'good' characters among blond-haired, blue-eyed characters."""
    bhbe = bhbe_col(df)
    good_bhbe = (df.loc[bhbe, 'Alignment'] == 'good').mean()
    return good_bhbe

def simulate_bhbe_null(df, N):
    """Simulate N test statistics under the null hypothesis."""
    bhbe = bhbe_col(df)
    good_proportion = (df['Alignment'] == 'good').mean()
    # Simulate the number of 'good' characters in the bhbe group under the null
    simulated_stats = np.random.binomial(n=bhbe.sum(), p=good_proportion, size=N) / bhbe.sum()
    return simulated_stats

def superheroes_p_value(df):
    """Compute the p-value and decide whether to reject the null hypothesis."""
    observed_stat = superheroes_observed_statistic(df)
    simulated_stats = simulate_bhbe_null(df, 100000)
    
    # Calculate the p-value based on the proportion of simulations >= observed_stat
    p_value = (simulated_stats >= observed_stat).mean()
    
    # Determine the hypothesis test result based on the 1% significance level
    decision = 'Reject' if p_value < 0.01 else 'Fail to reject'
    
    # Ensure the p-value is returned as a standard Python float, not np.float64
    return [float(p_value), decision]

In [28]:
# don't change this cell -- it is needed for the tests to work
superheroes_fp = Path('data') / 'superheroes.csv'
heroes = pd.read_csv(superheroes_fp, index_col=0)
bhbe_out = bhbe_col(heroes)
obs_stat_out = superheroes_observed_statistic(heroes)
simulate_bhbe_out = simulate_bhbe_null(heroes, 10)
pval_out = superheroes_p_value(heroes)

In [29]:
pval_out

[0.00011, 'Reject']

In [30]:
grader.check("q5")

q5 results: All test cases passed!

<div class="alert alert-block alert-danger" markdown="1">

**You cannot use `for`-loops in Part 1 (Time Series Data) or Part 2 (Hypothesis Testing), but you can use them in Part 3 (Permutation Testing).**

</div>

## Part 3: Permutation Testing

Recall, hypothesis tests answer questions of the form:

> I have a population distribution, and I have one sample. Does this sample look like it was drawn from the population?

While permutation tests answer questions of the form:

> I have two samples, but no information about any population distributions. Do these samples look like they were drawn from the same population?

Keep this in mind while working on this last part of the lab.

<br>

[Skittles](https://en.wikipedia.org/wiki/Skittles_(confectionery)) 🍬 are made in two locations in the United States: Yorkville, Illinois and Waco, Texas. In these factories, Skittles of different colors are made separately by different machines and combined/packaged into bags for sale. The **tab-separated file** `data/skittles.tsv` contains the contents of 468 bags of Skittles.

Throughout this question, we will compare the color distribution of Skittles between bags made in the Yorkville factory and bags made in the Waco factory. Most people have preferences for their favorite flavor, and there is a surprising amount of variation among the distribution of flavors in each bag.

Look at the variation by bag in the dataset below:

In [31]:
skittles_fp = Path('data') / 'skittles.tsv'
skittles = pd.read_csv(skittles_fp, sep='\t')
skittles.head()

,red,orange,yellow,green,purple,Factory
0,10,15,11,7,18,Yorkville
1,5,12,17,15,10,Yorkville
2,16,11,15,11,9,Waco
3,15,8,13,16,7,Waco
4,11,14,20,8,7,Waco


In [32]:
skittles.shape

(468, 6)

### Question 6 – Orange Skittles 🟠

First, you will investigate if the machine that mixes together the Skittles of different colors might favor one color over another. Use a permutation test to assess whether, on average, bags made in Yorkville have the same number of orange skittles as bags made in Waco. Do this by implementing the functions described below.

<br>

#### `diff_of_means`

Complete the implementation of the function `diff_of_means`, which takes in a DataFrame like `skittles` and returns the **absolute difference** between the **mean** number of orange Skittles per bag from Yorkville and the **mean** number of orange Skittles per bag from Waco.

<br>

#### `simulate_null`

Complete the implementation of the function `simulate_null`, which takes in a DataFrame like `skittles` and returns one simulated instance of the test statistic under the null hypothesis. Note that this will involve shuffling the `'Factory'` column!

<br>

#### `color_p_value`

Complete the implementation of the function `color_p_value`, which takes in a DataFrame like `skittles` and calculates the p-value for the permutation test using 1000 trials.

<br>

Plot the observed statistic, along with the histogram for the simulated distribution, to check your work.

***Note***: In all functions, the default argument for `col` is `'orange'`. Your functions should still work for any color so that you can call it in later questions.

In [33]:
def diff_of_means(data, col='orange'):
    yorkville_mean = data[data['Factory'] == 'Yorkville'][col].mean()
    waco_mean = data[data['Factory'] == 'Waco'][col].mean()
    return abs(yorkville_mean - waco_mean)

def simulate_null(data, col='orange'):
    shuffled_data = data.copy()
    shuffled_data['Factory'] = np.random.permutation(shuffled_data['Factory'])
    return diff_of_means(shuffled_data, col)

def color_p_value(data, col='orange', num_trials=1000):
    observed_diff = diff_of_means(data, col)
    simulated_diffs = [simulate_null(data, col) for _ in range(num_trials)]
    p_value = np.mean(np.array(simulated_diffs) >= observed_diff)
    return p_value

In [34]:
# don't change this cell -- it is needed for the tests to work
# cell may take about 1-2 minutes to execute to completion
skittles_fp = Path('data') / 'skittles.tsv'
skittles = pd.read_csv(skittles_fp, sep='\\t', engine='python')
q6_diff_of_means_out = diff_of_means(skittles)
q6_simulate_null_out = simulate_null(skittles)
q6_pval_out = color_p_value(skittles)

In [35]:
grader.check("q6")

q6 results: All test cases passed!

### Question 7 – Generalizing to all colors 🔴🟠🟡🟢🟣

While your `color_p_value` function used a default color of `'orange'`, it should also work for all other colors of Skittles, meaning you can run the same permutation test from Question 7 on all colors of Skittles. Call `color_p_value` on all colors of Skittles to find which colors differ the most between the two locations on average. 

Then, complete the implementation of the function `ordered_colors`, which returns a list of five ordered pairs, each of the form `('color', p_value)`. For example, your list might look like `[('pink', 0.000), ('brown', 0.025), ...]`. 

The list should be **hard-coded**, meaning that you should run your permutation tests in your notebook, not in your `.py` file. The list should also be sorted in **increasing order of p-value**. Make sure your p-values are rounded to **3 decimal places**.

Even though there is randomness in the color composition in each bag, this list gives the likelihood that the machines have a systematic, meaningful, difference in how they blend the colors in each bag.

In [36]:
here = [
        ('orange', color_p_value(skittles,"orange")),
        ('yellow', color_p_value(skittles,"yellow")),
        ('red', color_p_value(skittles,"red")),
        ('green', color_p_value(skittles,"green")),
        ('purple', color_p_value(skittles,"purple"))
    ]
for i in here:
    print(i)

('orange', np.float64(0.053))
('yellow', np.float64(0.0))
('red', np.float64(0.241))
('green', np.float64(0.504))
('purple', np.float64(0.977))


In [37]:
def ordered_colors():
    """Return a hard-coded list of ('color', p_value) pairs, sorted by p-value."""
    return [
        ('orange', (0.056)),
        ('yellow', (0.0)),
        ('red', (0.24)),
        ('green', (0.476)),
        ('purple', (0.976))
    ]


In [38]:
# don't change this cell -- it is needed for the tests to work
q7_out = ordered_colors()
q7_colors = {'green', 'orange', 'purple', 'red', 'yellow'}
q7_test_colors = [x[0] for x in q7_out]

In [39]:
grader.check("q7")

q7 results:
    q7 - 1 result:
        Test case passed!

    q7 - 2 result:
        Test case passed!

    q7 - 3 result:
        Test case passed!

    q7 - 4 result:
        Test case passed!

    q7 - 5 result:
        Trying:
            bool(np.isclose(q7_out[0][1], 0.0))
        Expecting:
            True
        **********************************************************************
        Line 1, in q7 4
        Failed example:
            bool(np.isclose(q7_out[0][1], 0.0))
        Expected:
            True
        Got:
            False

    q7 - 5 message: smallest pval

### Question 8 – Overall distributions 🏭

Now, suppose you would like to assess whether the two locations make similar amounts of each color overall. That is, suppose we:
* Combine and count up all the Skittles of each color that were made in Yorkville (e.g. 14303 total red skittles, 9091 total green skittles, etc.).
* Combine and count up all the Skittles of each color that were made in Waco.

Now, suppose you would like to assess whether the two locations make similar proportions of each color overall. That is, suppose we:
* Calculate the proportion of each Skittles color that were made in Yorkville (e.g. out of the 14704 skittles made, 19.8% of them are red,  18.9% of them are green, etc.).
* Calculate the proportion of each Skittles color that were made in Waco.

**Are these distributions of colors similar?** Is the variation among the bags due to each factory making different amounts of each color?

Use a permutation test to assess whether the distribution of colors of Skittles made in Yorkville is statistically significantly different than those made in Waco. Set a significance level (i.e. p-value cutoff) of 0.01 and determine whether you can reject a null hypothesis that answers the question above using a permutation test with 1000 trials. For your test statistic, use the **total variation distance (TVD)**.

Refer to the end of [Lecture 6](https://dsc80.com/resources/lectures/lec06/lec06.html#Permutation-testing-meets-TVD) to see an example of a [permutation test](https://www.inferentialthinking.com/chapters/12/Comparing_Two_Samples.html) that uses the [TVD](https://inferentialthinking.com/chapters/11/2/Multiple_Categories.html) as the test statistic. Some guidance:

- Our previous permutation tests have compared the mean number of (say) orange Skittles in Yorkville bags to the mean number number of orange Skittles in Waco bags. The role of shuffling was to randomly assign bags to Yorkville and Waco.
- In this permutation test, we are **still** shuffling to randomly assign bags to Yorkville and Waco. The only difference is that after we randomly assign each bag to a factory, we will compute the **distribution** of colors among the two factories and find the TVD between those two distributions.

**Your job**: Complete the implementation of the function `same_color_distribution`, which takes in no arguments and outputs a hard-coded **tuple** with the p-value and whether you `'Reject'` or `'Fail to reject'` the null hypothesis.

In [40]:
skittles

,red,orange,yellow,green,purple,Factory
0,10,15,11,7,18,Yorkville
1,5,12,17,15,10,Yorkville
2,16,11,15,11,9,Waco
3,15,8,13,16,7,Waco
4,11,14,20,8,7,Waco
...,...,...,...,...,...,...
463,11,11,12,13,11,Waco
464,17,10,8,11,12,Waco
465,9,14,12,10,15,Waco
466,12,14,11,10,10,Waco


In [41]:
# def calculate_tvd(p, q):
#     """Calculate the Total Variation Distance between two distributions."""
#     return 0.5 * np.sum(np.abs(p - q))

# def same_color_distribution():
#     # Load the data (assuming the data is in 'data/skittles.csv')
#     data = skittles

#     # Separate Skittles by factory
#     yorkville = data[data['Factory'] == 'Yorkville']
#     waco = data[data['Factory'] == 'Waco']

#     # Calculate the proportion of each color made by both factories
#     yorkville_proportions = yorkville['Color'].value_counts(normalize=True).sort_index()
#     waco_proportions = waco['Color'].value_counts(normalize=True).sort_index()

#     # Compute the observed TVD
#     observed_tvd = calculate_tvd(yorkville_proportions, waco_proportions)

#     # Perform permutation test with 1000 trials
#     combined = data.copy()
#     n_trials = 1000
#     tvd_values = np.zeros(n_trials)

#     for i in range(n_trials):
#         # Shuffle the factory labels
#         shuffled_labels = np.random.permutation(combined['Factory'])
#         combined['Shuffled Factory'] = shuffled_labels

#         # Compute new proportions for the shuffled labels
#         yorkville_shuffled = combined[combined['Shuffled Factory'] == 'Yorkville']
#         waco_shuffled = combined[combined['Shuffled Factory'] == 'Waco']

#         yorkville_shuffled_prop = yorkville_shuffled['Color'].value_counts(normalize=True).sort_index()
#         waco_shuffled_prop = waco_shuffled['Color'].value_counts(normalize=True).sort_index()

#         # Ensure both distributions have the same color order
#         tvd_values[i] = calculate_tvd(yorkville_shuffled_prop, waco_shuffled_prop)

#     # Calculate the p-value
#     p_value = (tvd_values >= observed_tvd).mean()

#     # Determine whether to reject the null hypothesis
#     decision = 'Reject' if p_value < 0.01 else 'Fail to reject'

#     # Return a hard-coded result
#     return (round(p_value, 3), decision)

In [42]:
skittles

,red,orange,yellow,green,purple,Factory
0,10,15,11,7,18,Yorkville
1,5,12,17,15,10,Yorkville
2,16,11,15,11,9,Waco
3,15,8,13,16,7,Waco
4,11,14,20,8,7,Waco
...,...,...,...,...,...,...
463,11,11,12,13,11,Waco
464,17,10,8,11,12,Waco
465,9,14,12,10,15,Waco
466,12,14,11,10,10,Waco


In [43]:
# def calculate_tvd(p, q):
#     """Calculate the Total Variation Distance between two distributions."""
#     return 0.5 * np.sum(np.abs(p - q))

# def same_color_distribution():
#     # Load the data
#     # data = skittles
#     data = pd.read_csv('data/skittles.tsv')

    
#     # Aggregate the counts of each color by factory
#     yorkville_totals = data[data['Factory'] == 'Yorkville'].iloc[:, :-1].sum()
#     waco_totals = data[data['Factory'] == 'Waco'].iloc[:, :-1].sum()

#     # Compute proportions of each color for both factories
#     yorkville_proportions = yorkville_totals / yorkville_totals.sum()
#     waco_proportions = waco_totals / waco_totals.sum()

#     # Calculate the observed TVD between the two factories
#     observed_tvd = calculate_tvd(yorkville_proportions, waco_proportions)

#     # Perform permutation test with 1000 trials
#     n_trials = 1000
#     tvd_values = np.zeros(n_trials)

#     for i in range(n_trials):
#         # Shuffle the factory labels
#         shuffled_labels = np.random.permutation(data['Factory'])

#         # Assign new factory labels and aggregate by the shuffled labels
#         yorkville_shuffled_totals = data[shuffled_labels == 'Yorkville'].iloc[:, :-1].sum()
#         waco_shuffled_totals = data[shuffled_labels == 'Waco'].iloc[:, :-1].sum()

#         # Compute new proportions for shuffled data
#         yorkville_shuffled_prop = yorkville_shuffled_totals / yorkville_shuffled_totals.sum()
#         waco_shuffled_prop = waco_shuffled_totals / waco_shuffled_totals.sum()

#         # Compute TVD for the shuffled data
#         tvd_values[i] = calculate_tvd(yorkville_shuffled_prop, waco_shuffled_prop)

#     # Calculate the p-value
#     p_value = (tvd_values >= observed_tvd).mean()

#     # Determine whether to reject the null hypothesis
#     decision = 'Reject' if p_value < 0.01 else 'Fail to reject'

#     # Return a hard-coded result (rounded p-value and decision)
#     return (round(p_value, 3), decision)

In [44]:

def calculate_tvd(p, q):
    """Calculate the Total Variation Distance between two distributions."""
    return 0.5 * np.sum(np.abs(p - q))

def same_color_distribution():
    # Load the data and ensure correct parsing
    data = pd.read_csv('data/skittles.tsv', sep='\t')
    
    # Check column names and clean up any whitespace
    data.columns = data.columns.str.strip()

    # Aggregate the counts of each color by factory
    yorkville_totals = data[data['Factory'] == 'Yorkville'].iloc[:, :-1].sum()
    waco_totals = data[data['Factory'] == 'Waco'].iloc[:, :-1].sum()

    # Compute proportions of each color for both factories
    yorkville_proportions = yorkville_totals / yorkville_totals.sum()
    waco_proportions = waco_totals / waco_totals.sum()

    # Calculate the observed TVD between the two factories
    observed_tvd = calculate_tvd(yorkville_proportions, waco_proportions)

    # Perform permutation test with 1000 trials
    n_trials = 1000
    tvd_values = np.zeros(n_trials)

    for i in range(n_trials):
        # Shuffle the factory labels
        shuffled_labels = np.random.permutation(data['Factory'])

        # Assign new factory labels and aggregate by the shuffled labels
        yorkville_shuffled_totals = data[shuffled_labels == 'Yorkville'].iloc[:, :-1].sum()
        waco_shuffled_totals = data[shuffled_labels == 'Waco'].iloc[:, :-1].sum()

        # Compute new proportions for shuffled data
        yorkville_shuffled_prop = yorkville_shuffled_totals / yorkville_shuffled_totals.sum()
        waco_shuffled_prop = waco_shuffled_totals / waco_shuffled_totals.sum()

        # Compute TVD for the shuffled data
        tvd_values[i] = calculate_tvd(yorkville_shuffled_prop, waco_shuffled_prop)

    # Calculate the p-value
    p_value = (tvd_values >= observed_tvd).mean()

    # Determine whether to reject the null hypothesis
    decision = 'Reject' if p_value < 0.01 else 'Fail to reject'

    # Return a hard-coded result (rounded p-value and decision)
    return (round(p_value, 3), decision)


In [45]:
# don't change this cell -- it is needed for the tests to work
q8_out = same_color_distribution()

In [46]:
grader.check("q8")

q8 results:
    q8 - 1 result:
        Test case passed!

    q8 - 2 result:
        Test case passed!

    q8 - 3 result:
        Trying:
            q8_out[1] in ['Fail to Reject', 'Reject']
        Expecting:
            True
        **********************************************************************
        Line 1, in q8 2
        Failed example:
            q8_out[1] in ['Fail to Reject', 'Reject']
        Expected:
            True
        Got:
            False

    q8 - 3 message: wrong output type at index 1

    q8 - 4 result:
        Test case passed!

    q8 - 5 result:
        Test case passed!

### Question 9 – Permutation testing vs. hypothesis testing 🧪

In each of the following scenarios, decide  whether  a  permutation test is appropriate to determine if there is a  significant difference between the quantities described. If a permutation test is appropriate, mark `'P'`. Otherwise, mark `'H'`.

Record your answers in the function `perm_vs_hyp`, which outputs a list of length 5, containing the values `'P'` and `'H'`.

1. Compare the DSC 80 pass rate between second years and third years who take the class.
2. Compare the proportion of Data Science majors who have completed DSC 80 and the proportion of Data Science minors who have completed DSC 80.
3. Compare the proportion of students who have iPhones to the proportion of students who have Android phones (for simplicity, assume that all students either have an iPhone or an Android).
4. In DSC 80, we ask all students whether they liked DSC 40A or DSC 40B more. Compare the proportion of students who preferred DSC 40A to the proportion who preferred DSC 40B.
5. A sales company has two branches, one in city A and one in city B. You want to compare the amount of sales revenue brought in by employees at the two branches.

***Hint***: Think about the type of data you would collect in each case, and how you would simulate new data under the null hypothesis. It will be useful to refer to the explanation at the start of Part 3.

In [50]:
def perm_vs_hyp():
    return ['P', 'P', 'P', 'H', 'P']


In [51]:
# don't change this cell -- it is needed for the tests to work
q9_out = perm_vs_hyp()

In [52]:
grader.check("q9")

q9 results: All test cases passed!

## Congratulations! You're done Lab 4! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q2 q4
```

will run the `grader.check` cells for Questions 1, 2, and 4 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [49]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!

q6 results: All test cases passed!

q7 results:
    q7 - 1 result:
        Test case passed!

    q7 - 2 result:
        Test case passed!

    q7 - 3 result:
        Test case passed!

    q7 - 4 result:
        Test case passed!

    q7 - 5 result:
        Trying:
            bool(np.isclose(q7_out[0][1], 0.0))
        Expecting:
            True
        **********************************************************************
        Line 1, in q7 4
        Failed example:
            bool(np.isclose(q7_out[0][1], 0.0))
        Expected:
            True
        Got:
            False

    q7 - 5 message: smallest pval

q8 results:
    q8 - 1 result:
        Test case passed!

    q8 - 2 result:
        Test case passed!

    q8 - 3 result:
        Trying:
            q8_out[1] in ['Fail to Reject', 'Reject']